# Exploring <i>"Atomic Habits" By James Clear </i> using <b> Langchain and SingleStore

In [7]:
!pip install langchain --quiet
!pip install --upgrade openai==0.28.1 --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
!pip install singlestoredb --quiet
!pip install tiktoken --quiet
!pip install --upgrade unstructured==0.10.14 --quiet

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Using the <b>Edit Firewall</b> button in the top right, add the following to the SingleStoreDB Cloud notebook firewall:
            <ul style="list-style: none;">
                <li><b>ztcprep.com.</b></li>
            </ul>
        </p>
    </div>
</div>

In [13]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://ztcprep.com/library/story/Atomic_Habits/Atomic_Habits_(www.ztcprep.com).pdf")

data = loader.load()

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 467900 characters in your document


In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 256 pages


In [17]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [18]:
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [19]:
%%sql


CREATE DATABASE IF NOT EXISTS pdf_db;

USE pdf_db;

DROP TABLE IF EXISTS pdf_atomic_habits;

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SingleStoreDB to make connections to the selected database.</p>
    </div>
</div>

In [20]:
from langchain.vectorstores import SingleStoreDB

docsearch = SingleStoreDB.from_documents(
    texts,
    embedding,
    table_name = "pdf_atomic_habits",
)

## Prompt1:Who is the author of this book?

In [26]:
query_text = "Who is the author?"

docs = docsearch.similarity_search(query_text)

#print(docs[0].page_content)

In [27]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

The author of the document is James Clear.


## Prompt 2: What is the Goldilocks rule? 

In [31]:
query_text = "What is the Goldilocks rule? "

docs = docsearch.similarity_search(query_text)

print(docs[0].page_content)

Martin’s	story	offers	a	fascinating	perspective	on	what	it	takes	to stick	with	habits	for	the	long	run.	Comedy	is	not	for	the	timid.	It	is hard	to	imagine	a	situation	that	would	strike	fear	into	the	hearts	of more	people	than	performing	alone	on	stage	and	failing	to	get	a	single laugh.	And	yet	Steve	Martin	faced	this	fear	every	week	for	eighteen years.	In	his	words,	“10	years	spent	learning,	4	years	spent	refining, and	4	years	as	a	wild	success.”

Why	is	it	that	some	people,	like	Martin,	stick	with	their	habits— whether	practicing	jokes	or	drawing	cartoons	or	playing	guitar—while most	of	us	struggle	to	stay	motivated?	How	do	we	design	habits	that pull	us	in	rather	than	ones	that	fade	away?	Scientists	have	been studying	this	question	for	many	years.	While	there	is	still	much	to learn,	one	of	the	most	consistent	findings	is	that	the	way	to	maintain motivation	and	achieve	peak	levels	of	desire	is	to	work	on	tasks	of	“just manageable	difficulty.”

The	human	brain	loves	a	challenge,	but	onl

In [32]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

The Goldilocks Rule states that humans experience peak motivation when working on tasks that are right on the edge of their current abilities. Not too hard. Not too easy. Just right. This concept suggests that people thrive when challenged, but only if these challenges are within their optimal zone of difficulty. It's a balance between not getting bored with tasks that are too simple, and not getting demotivated with tasks that are too complex.


## Prompt 3:How can we  stop procrastinating.

In [42]:
query_text = "How can we  stop procrastinating? "

docs = docsearch.similarity_search(query_text)

print(docs[0].page_content)

Many	people	think	they	lack	motivation	when	what	they	really	lack is	clarity.	It	is	not	always	obvious	when	and	where	to	take	action.	Some people	spend	their	entire	lives	waiting	for	the	time	to	be	right	to	make an	improvement.

www.ztcprep.com

Once	an	implementation	intention	has	been	set,	you	don’t	have	to wait	for	inspiration	to	strike.	Do	I	write	a	chapter	today	or	not?	Do	I meditate	this	morning	or	at	lunch?	When	the	moment	of	action occurs,	there	is	no	need	to	make	a	decision.	Simply	follow	your predetermined	plan.

The	simple	way	to	apply	this	strategy	to	your	habits	is	to	fill out	this	sentence:

I	will	[BEHAVIOR]	at	[TIME]	in	[LOCATION].

Meditation.	I	will	meditate	for	one	minute	at	7	a.m.	in	my	kitchen.

Studying.	I	will	study	Spanish	for	twenty	minutes	at	6	p.m.	in	my bedroom.

Exercise.	I	will	exercise	for	one	hour	at	5	p.m.	in	my	local	gym.

Marriage.	I	will	make	my	partner	a	cup	of	tea	at	8	a.m.	in	the kitchen.

If	you	aren’t	sure	when	to	start	your	habit,	try	the	first

In [43]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

To stop procrastinating, you can start by setting clear and specific goals, known as implementation intentions. This means deciding ahead of time what action you will take, when, and where. Fill out the sentence: "I will [Behavior] at [Time] in [Location]." For example, "I will study Spanish for 20 minutes at 6 p.m. in my bedroom" or "I will exercise for one hour at 5 p.m. in my local gym."

By being specific about what you want and how you will achieve it, it becomes easier to stick to your plan and avoid distractions or delays. Start creating these implementation intentions on the first day of a week, month, or year when motivation tends to be higher.

Above all, gaining clarity on what exactly you need to do can help avoid the trap of procrastination. When your plans are vague, it's easy to rationalize and put off tasks. But when you know exactly what needs to be done and when, it's easier to take action.


In [90]:
query_text = "How can change a bad habit? "

docs = docsearch.similarity_search(query_text)

print(docs[0].page_content)

if	you	were	watching	someone	else.	Oh,	how	interesting	that	they would	do	such	a	thing.	If	you	binge-eat,	simply	notice	that	you	are eating	more	calories	than	you	should.	If	you	waste	time	online,	notice that	you	are	spending	your	life	in	a	way	that	you	do	not	want	to.

The	first	step	to	changing	bad	habits	is	to	be	on	the	lookout	for them.	If	you	feel	like	you	need	extra	help,	then	you	can	try	Pointing- and-Calling	in	your	own	life.	Say	out	loud	the	action	that	you	are thinking	of	taking	and	what	the	outcome	will	be.	If	you	want	to	cut back	on	your	junk	food	habit	but	notice	yourself	grabbing	another cookie,	say	out	loud,	“I’m	about	to	eat	this	cookie,	but	I	don’t	need	it. Eating	it	will	cause	me	to	gain	weight	and	hurt	my	health.”

Hearing	your	bad	habits	spoken	aloud	makes	the	consequences

seem	more	real.	It	adds	weight	to	the	action	rather	than	letting yourself	mindlessly	slip	into	an	old	routine.	This	approach	is	useful even	if	you’re	simply	trying	to	remember	a	task	on	your	to-d

In [91]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

To change a bad habit, you first need to become aware of your actions. To become more aware, you can use strategies like Pointing-and-Calling or the Habits Scorecard. Pointing-and-Calling involves saying out loud the action you're about to take and what its outcome will be, which makes the consequences seem more real. For example, if you're trying to eat less junk food, you would voice out when you're about to eat a cookie and mention the negative impact it could have, like weight gain or harmful effects on health. 

On the other hand, the Habits Scorecard involves recognizing your habits and acknowledging the triggers. Having knowledge of these cues will then allow you to respond in a way that benefits you. These steps will help you understand your habits better and provide the foundation for modifying them.


Enter or Try with other prompts in the query_text variable and Run the subsequent cell to view the summarization for the given prompt.
<b>Read Books Be Happy!